<a href="https://colab.research.google.com/github/ashaaher/Emerging-Technologies/blob/master/Assignment%204/Reptile_Sine_wave_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sine wave Regression Using Reptile

Importing necessary libraries 

In [ ]:
import tensorflow as tf
import numpy as np

Generating Data Points

Defining a function called sample_points for generating (x,y) pairs. 

In [ ]:
def sample_points(k):
    
    num_points = 100
    
    #amplitude
    amplitude = np.random.uniform(low=0.1, high=5.0)
    
    #phase
    phase = np.random.uniform(low=0, high=np.pi)

    x = np.linspace(-5, 5, num_points)

    #y = a*sin(x+b)
    y = amplitude * np.sin(x + phase)
    
    #sample k data points
    sample = np.random.choice(np.arange(num_points), size=k)
    
    return (x[sample], y[sample])

In [ ]:
x, y = sample_points(5)
print x
print y

[ 1.96969697 -2.17171717  3.18181818  1.06060606  2.67676768]
[-0.97122995 -0.19398046 -1.1409783   0.07730279 -1.29343206]


Two Layered Neural Network.

In [ ]:
# resetting the tensorflow graph.
tf.reset_default_graph()

Initializing network parameters.

In [ ]:
num_hidden = 64
num_classes = 1
num_feature = 1

Defining the placeholders for input and output.

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, num_feature])
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

Randomly initialize model parameters, 

In [ ]:
w1 = tf.Variable(tf.random_uniform([num_feature, num_hidden]))
b1 = tf.Variable(tf.random_uniform([num_hidden]))

w2 = tf.Variable(tf.random_uniform([num_hidden, num_classes]))
b2 = tf.Variable(tf.random_uniform([num_classes]))

feedforward operation to predict the output

In [ ]:
#layer 1
z1 = tf.matmul(X, w1) + b1
a1 = tf.nn.tanh(z1)

#output layer
z2 = tf.matmul(a1, w2) + b2
Yhat = tf.nn.tanh(z2)

Using loss function-Mean Squared 

In [ ]:
loss_function = tf.reduce_mean(tf.square(Yhat - Y))

Using Adam Optimizer to optimise the loss

In [ ]:
optimizer = tf.train.AdamOptimizer(1e-2).minimize(loss_function)

Initializing tensorflow variables.

In [ ]:
init = tf.global_variables_initializer()

# Reptile


In [ ]:
#number of epochs i.e training iterations
num_epochs = 100


#number of samples i.e number of shots
num_samples = 50  

#number of tasks
num_tasks = 2

#number of times we want to perform optimization
num_iterations = 10


#mini btach size
mini_batch = 10  

In [ ]:
#start the tensorflow session
with tf.Session() as sess:
    
    sess.run(init)
    
    for e in range(num_epochs):
        
        #for each task in batch of tasks
        for task in range(num_tasks):

            #get the initial parameters of the model
            old_w1, old_b1, old_w2, old_b2 = sess.run([w1, b1, w2, b2,])

            #sample x and y
            x_sample, y_sample = sample_points(num_samples)


            #for some k number of iterations perform optimization on the task
            for k in range(num_iterations):

                #get the minibatch x and y
                for i in range(0, num_samples, mini_batch):

                    #sample mini batch of examples 
                    x_minibatch = x_sample[i:i+mini_batch]
                    y_minibatch = y_sample[i:i+mini_batch]


                    train = sess.run(optimizer, feed_dict={X: x_minibatch.reshape(mini_batch,1), 
                                                           Y: y_minibatch.reshape(mini_batch,1)})

            #get the updated model parameters after several iterations of optimization
            new_w1, new_b1, new_w2, new_b2 = sess.run([w1, b1, w2, b2])

            #Now we perform meta update

            #i.e theta = theta + epsilon * (theta_star - theta)

            epsilon = 0.1

            updated_w1 = old_w1 + epsilon * (new_w1 - old_w1) 
            updated_b1 = old_b1 + epsilon * (new_b1 - old_b1) 

            updated_w2 = old_w2 + epsilon * (new_w2 - old_w2) 
            updated_b2 = old_b2 + epsilon * (new_b2 - old_b2) 


            #update the model parameter with new parameters
            w1.load(updated_w1, sess)
            b1.load(updated_b1, sess)

            w2.load(updated_w2, sess)
            b2.load(updated_b2, sess)

        if e%10 == 0:
            loss = sess.run(loss_function, feed_dict={X: x_sample.reshape(num_samples,1), Y: y_sample.reshape(num_samples,1)})

            print "Epoch {}: Loss {}\n".format(e,loss)             
            print 'Updated Model Parameter Theta\n'
            print 'Sampling Next Batch of Tasks \n'
            print '---------------------------------\n'

Epoch 0: Loss 4.162981987

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 10: Loss 1.52488529682

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 20: Loss 1.74668705463

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 30: Loss 0.955383658409

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 40: Loss 9.85152721405

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 50: Loss 3.62764883041

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 60: Loss 5.67536497116

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 70: Loss 1.39854609966

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------

Reference : https://github.com/sudharsan13296/Hands-On-Meta-Learning-With-Python/blob/master/07.%20Meta-SGD%20and%20Reptile%20Algorithms/7.7%20Sine%20wave%20Regression%20Using%20Reptile.ipynb